In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import requests
from tqdm.auto import tqdm
from scipy.stats import mode 
from pprint import pprint
import warnings
warnings.filterwarnings("ignore")

from rectools import Columns
from rectools.dataset import Interactions, Dataset
from rectools.metrics import Precision, Recall, MeanInvUserFreq, Serendipity, calc_metrics, MAP
from rectools.model_selection import TimeRangeSplitter
from rectools.models import ImplicitItemKNNWrapperModel, PopularModel, PureSVDModel

from implicit.nearest_neighbours import CosineRecommender, TFIDFRecommender, BM25Recommender
from implicit.als import AlternatingLeastSquares

import dill

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)

In [2]:
N = 20

# 🎬 Get KION dataset 

<a href="https://ods.ai/competitions/competition-recsys-21/data"> Dataset description [ru] </a>

In [3]:
# download dataset by chunks
url = "https://storage.yandexcloud.net/itmo-recsys-public-data/kion_train.zip"

req = requests.get(url, stream=True)

with open('../data/kion_train.zip', "wb") as fd:
    total_size_in_bytes = int(req.headers.get('Content-Length', 0))
    progress_bar = tqdm(desc='kion dataset download', total=total_size_in_bytes, unit='iB', unit_scale=True)
    for chunk in req.iter_content(chunk_size=2 ** 20):
        progress_bar.update(len(chunk))
        fd.write(chunk)

In [4]:
!cd ../data && unzip kion_train.zip && cd -

# EDA

In [3]:
interactions = pd.read_csv('../data/kion_train/interactions.csv')
users = pd.read_csv('../data/kion_train/users.csv')
items = pd.read_csv('../data/kion_train/items.csv')

In [4]:
# rename columns, convert timestamp
interactions.rename(columns={'last_watch_dt': Columns.Datetime,
                            'total_dur': Columns.Weight}, 
                    inplace=True) 

interactions['datetime'] = pd.to_datetime(interactions['datetime'])

## interactions

In [5]:
pd.concat([interactions.head(), interactions.tail()])

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0
5476246,648596,12225,2021-08-13,76,0.0
5476247,546862,9673,2021-04-13,2308,49.0
5476248,697262,15297,2021-08-20,18307,63.0
5476249,384202,16197,2021-04-19,6203,100.0
5476250,319709,4436,2021-08-15,3921,45.0


In [6]:
print(f"Interactions dataframe shape: {interactions.shape}")
print(f"Unique users in interactions: {interactions['user_id'].nunique():_}")
print(f"Unique items in interactions: {interactions['item_id'].nunique():_}")

Interactions dataframe shape: (5476251, 5)
Unique users in interactions: 962_179
Unique items in interactions: 15_706


In [7]:
max_date = interactions['datetime'].max()
min_date = interactions['datetime'].min()

print(f"min date in interactions: {min_date}")
print(f"max date in interactions: {max_date}")

min date in interactions: 2021-03-13 00:00:00
max date in interactions: 2021-08-22 00:00:00


In [8]:
interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5476251 entries, 0 to 5476250
Data columns (total 5 columns):
 #   Column       Dtype         
---  ------       -----         
 0   user_id      int64         
 1   item_id      int64         
 2   datetime     datetime64[ns]
 3   weight       int64         
 4   watched_pct  float64       
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 208.9 MB


In [9]:
rec_interactions = Interactions(interactions)

## users

In [10]:
pd.concat([users.head(), users.tail()])

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
2,1047345,age_45_54,income_40_60,Ж,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0
840192,339025,age_65_inf,income_0_20,Ж,0
840193,983617,age_18_24,income_20_40,Ж,1
840194,251008,NaN,NaN,NaN,0
840195,590706,NaN,NaN,Ж,0
840196,166555,age_65_inf,income_20_40,Ж,0


In [11]:
print(f"Users dataframe shape {users.shape}")
print(f"Unique users: {users['user_id'].nunique():_}")

Users dataframe shape (840197, 5)
Unique users: 840_197


## items

In [12]:
pd.concat([items.head(3), items.tail(3)])

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Грандинетти, Джеральдин Чаплин, Елена Анайя, Каэтано Велозо, Леонор Уотлинг, Лола Дуэньяс, Лолес Леон, Малу Айродо, Мариола Фуэнтес, Пас Вега, Пина Бауш, Ро...",Мелодрама легендарного Педро Альмодовара «Поговори с ней» в 2003 году получила премию «Оскар» за лучший сценарий. Журналист Марко берет интервью у знаменитой женщины-тореро Лидии и вскоре влюбляе...,"Поговори, ней, 2002, Испания, друзья, любовь, сильные, женщины, преодоление, трудностей, отношения, дружба, отношения, паре, отношения, мужчины, женщины, романтические, отношения, потеря, близких,..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон Манцукас, Джон Глейсер, Карл Грин, Кристен Риттер, Лэнс Реддик, Морис Комт, Патрик Кернс, Ребекка Коллинз, Роза Салазар, Росс П. Кук, Стеффи Гроут, Ти...","Уморительная современная комедия на популярную тему о том, как не надо отмечать мальчишник. Главный герой усвоил, что не надо звать на свадьбу своего друга Джейсона, из-за которого он вместо сваде...","Голые, перцы, 2014, США, друзья, свадьбы, преодоление, трудностей, расставания, отношения, дружба, риск, недоразумение, мужская, дружба, мальчишники, девичники"
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман, Дэн Риззуто, Кендес Илэйн Калтраро, Кит Джардин, Лекса Дойг, Майкл Джей Уайт, Майкл Шэнкс, Майкл Эклунд, Питер Брайант, Питер Кент, Стив Бачич, Стив ...","Профессиональный рестлер Стив Остин («Все или ничего») и темнокожий мачо Майкл Джей Уайт («Темный рыцарь») в интригующем криминальном боевике. В центре сюжета – команда спецназовцев, которая оказа...","Тактическая, сила, 2011, Канада, бандиты, гангстеры, преступления, преодоление, трудностей, убийства, убийцы, настоящие, мужчины, риск, недоразумение, силы, правопорядка, борьба, за, выживание, сп..."
15960,10632,series,Сговор,Hassel,2017.0,"драмы, триллеры, криминал",Россия,0.0,18.0,NaN,"Эшреф Рейбрук, Амир Камдин, Эрик Эгер","Ола Рапас, Алиетт Офейм, Уильма Лиден, Шанти Рони, Тома Холмин","Криминальная драма по мотивам романов о шведском детективе Роланде Хасселе. Средь бела дня убит полицейский, и нити в этом деле ведут прямо в коридоры власти. Расследованием занимается детектив Ха...","Сговор, 2017, Россия"
15961,4538,series,Среди камней,Darklands,2019.0,"драмы, спорт, криминал",Россия,0.0,18.0,NaN,"Марк О’Коннор, Конор МакМахон","Дэйн Уайт О’Хара, Томас Кэйн-Бирн, Джудит Родди, Марк О’Халлоран, Джимми Смоллхорн","Семнадцатилетний Дэмиен мечтает вырваться за пределы своего района и стать профессиональным бойцом. Когда его кумир и старший брат исчезает, парень попадает в чуждый ему мир насилия, наркотиков и ...","Среди, камней, 2019, Россия"
15962,3206,series,Гоша,NaN,2019.0,комедии,Россия,0.0,16.0,NaN,Михаил Миронов,"Мкртыч Арзуманян, Виктория Рунцова","Добродушный Гоша не может выйти из дома, чтобы не попасть в нелепую и курьёзную историю. Но даже неудачники мечтают о любви, и наш герой — не исключение, ведь оптимизма ему не занимать.","Гоша, 2019, Россия"


In [13]:
print(f"Items dataframe shape {items.shape}")
print(f"Unique item_id: {items['item_id'].nunique():_}")

Items dataframe shape (15963, 14)
Unique item_id: 15_963


# Define Blending model

In [14]:
class BlendingModel():
    
    def __init__(self, model1, model2):
        self._model1 = model1
        self._model2 = model2
        
    def fit(self, X):
        self._model1.fit(X)
        self._model2.fit(X)
    
    def recommend(self, *args, **kwargs):
        reco1 = self._model1.recommend(*args, **kwargs)
        reco2 = self._model2.recommend(*args, **kwargs)
        mreco = pd.concat((reco1,reco2)).sort_values(["user_id", "rank"])
        mreco = mreco.drop_duplicates(subset=["user_id", "item_id"], keep="first")
        reco_groups = mreco.groupby("user_id")
        reco = reco_groups.head(kwargs["k"])
        reco["rank"] = np.concatenate([list(range(x)) for x in reco.groupby("user_id")["user_id"].count()])
        return reco

# Split

In [15]:
# setting for cv 
n_folds = 7
unit = "W"
n_units = 1

last_date = interactions[Columns.Datetime].max().normalize()
start_date = last_date - pd.Timedelta(n_folds * n_units + 1, unit=unit)  
print(f"Start date and last date of the test fold: {start_date, last_date}")

Start date and last date of the test fold: (Timestamp('2021-06-27 00:00:00'), Timestamp('2021-08-22 00:00:00'))


### Test fold borders

In [16]:
periods = n_folds + 1
freq = f"{n_units}{unit}"
print(
    f"start_date: {start_date}\n"
    f"last_date: {last_date}\n"
    f"periods: {periods}\n"
    f"freq: {freq}\n"
)
    
date_range = pd.date_range(start=start_date, periods=periods, freq=freq, tz=last_date.tz)
print(f"Test fold borders: {date_range.values.astype('datetime64[D]')}")

# generator of folds
cv = TimeRangeSplitter(
    date_range=date_range,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)
print(f"Real number of folds: {cv.get_n_splits(rec_interactions)}")

start_date: 2021-06-27 00:00:00
last_date: 2021-08-22 00:00:00
periods: 8
freq: 1W

Test fold borders: ['2021-06-27' '2021-07-04' '2021-07-11' '2021-07-18' '2021-07-25'
 '2021-08-01' '2021-08-08' '2021-08-15']
Real number of folds: 7


In [17]:
metrics = {
    "prec@10": Precision(k=10),
    "recall@10": Recall(k=10),
    "map@10": MAP(k=10),
    "novelty": MeanInvUserFreq(k=10),
    "serendipity": Serendipity(k=10),
}

In [18]:
# few simple models to compare
models = {
    "cosine_itemknn": CosineRecommender(K=N),
    "tfidf_itemknn": TFIDFRecommender(K=N),
    "bm25_itemknn": BM25Recommender(K=N),
    "pure_svd": PureSVDModel(),
    "blend_cosine_tfidf": BlendingModel(
        ImplicitItemKNNWrapperModel(CosineRecommender(K=N)),
        ImplicitItemKNNWrapperModel(TFIDFRecommender(K=N)),
    ),
    "blend_cosine_bm25": BlendingModel(
        ImplicitItemKNNWrapperModel(CosineRecommender(K=N)),
        ImplicitItemKNNWrapperModel(BM25Recommender(K=N)),
    ),
    "blend_cosine_pure_svd": BlendingModel(
        ImplicitItemKNNWrapperModel(CosineRecommender(K=N)),
        PureSVDModel(),
    ),
    "blend_tfidf_bm25": BlendingModel(
        ImplicitItemKNNWrapperModel(TFIDFRecommender(K=N)),
        ImplicitItemKNNWrapperModel(BM25Recommender(K=N)),
    ),
    "blend_tfidf_pure_svd": BlendingModel(
        ImplicitItemKNNWrapperModel(TFIDFRecommender(K=N)),
        PureSVDModel(),
    ),
    "blend_bm25_pure_svd": BlendingModel(
        ImplicitItemKNNWrapperModel(BM25Recommender(K=N)),
        PureSVDModel(),
    ),
}

# Model training by fold

In [19]:
%%time

results = []

fold_iterator = cv.split(rec_interactions, collect_fold_stats=True)

for i_fold, (train_ids, test_ids, fold_info) in tqdm(enumerate(fold_iterator), total=cv.get_n_splits(rec_interactions)):
    print(f"\n==================== Fold {i_fold}")
    pprint(fold_info)

    df_train = interactions.iloc[train_ids].copy()
    df_test = interactions.iloc[test_ids][Columns.UserItem].copy()

    interactions_train  = Dataset.construct(
        interactions_df=df_train,
        user_features_df=None,
        item_features_df=None
    )
    
    catalog = df_train[Columns.Item].unique()
    
    for model_name, model in models.items():
        if model_name == "pure_svd":
            model = model
        elif model_name.startswith("blend_"):
            modle = model
        else:
            model = ImplicitItemKNNWrapperModel(model=model)
        model.fit(interactions_train)
        
        recs_itemknn = model.recommend(
            df_test['user_id'].unique(), 
            dataset=interactions_train, 
            k=N, 
            filter_viewed=False  # False - same items to every user
        )
        metric_values = calc_metrics(
            metrics,
            reco=recs_itemknn,
            interactions=df_test,
            prev_interactions=df_train,
            catalog=catalog,
        )
    
        fold = {"fold": i_fold, "model": model_name}
        fold.update(metric_values)
        pprint(fold)
        results.append(fold)
        

  0%|          | 0/7 [00:00<?, ?it/s]


==================== Fold 0
{'End date': Timestamp('2021-07-04 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-06-27 00:00:00', freq='W-SUN'),
 'Test': 237414,
 'Test items': 5947,
 'Test users': 98930,
 'Train': 2533586,
 'Train items': 14092,
 'Train users': 536802}
{'fold': 0,
 'map@10': 0.03075359459250865,
 'model': 'cosine_itemknn',
 'novelty': 8.560670884128767,
 'prec@10': 0.02810573132517942,
 'recall@10': 0.17016937028416568,
 'serendipity': 3.5860072345484765e-05}
{'fold': 0,
 'map@10': 0.03762342914064018,
 'model': 'tfidf_itemknn',
 'novelty': 7.207676609557101,
 'prec@10': 0.03577580107146467,
 'recall@10': 0.20843044146161446,
 'serendipity': 5.41154388287347e-05}
{'fold': 0,
 'map@10': 0.07084579142326884,
 'model': 'bm25_itemknn',
 'novelty': 4.429813839871021,
 'prec@10': 0.060503386232689776,
 'recall@10': 0.347113748952724,
 'serendipity': 2.134204136402579e-05}
{'fold': 0,
 'map@10': 0.07638439498385285,
 'model': 'pure_svd',
 'novelty': 5.864203775918098

# 👌 Metrics 

`Metrics by fold`



In [20]:
df_metrics = pd.DataFrame(results)
df_metrics

,fold,model,prec@10,recall@10,map@10,novelty,serendipity
0,0,cosine_itemknn,0.028106,0.170169,0.030754,8.560671,0.000036
1,0,tfidf_itemknn,0.035776,0.208430,0.037623,7.207677,0.000054
2,0,bm25_itemknn,0.060503,0.347114,0.070846,4.429814,0.000021
3,0,pure_svd,0.045365,0.277388,0.076384,5.864204,0.000012
4,0,blend_cosine_tfidf,0.034223,0.200607,0.044698,8.064863,0.000047
...,...,...,...,...,...,...,...
65,6,blend_cosine_bm25,0.029396,0.156011,0.051011,6.055953,0.000044
66,6,blend_cosine_pure_svd,0.022850,0.124286,0.054601,6.440598,0.000039
67,6,blend_tfidf_bm25,0.032134,0.169321,0.055993,5.428684,0.000065
68,6,blend_tfidf_pure_svd,0.025901,0.139440,0.058389,5.852272,0.000058


In [21]:
df_metrics.to_pickle("../data/df_metrics.pickle")

## Metrics mean by fold
`we can compare two models`

In [22]:
df_metrics.groupby('model').mean()[metrics.keys()]

,prec@10,recall@10,map@10,novelty,serendipity
model,,,,,
blend_bm25_pure_svd,0.038568,0.214104,0.090257,4.805587,0.000022
blend_cosine_bm25,0.036278,0.202373,0.063879,6.181315,0.000037
blend_cosine_pure_svd,0.028058,0.160783,0.068000,6.660195,0.000034
blend_cosine_tfidf,0.027161,0.153928,0.048890,7.640792,0.000069
blend_tfidf_bm25,0.039191,0.216983,0.069049,5.567993,0.000054
blend_tfidf_pure_svd,0.031021,0.175763,0.071740,6.099870,0.000049
bm25_itemknn,0.042180,0.231087,0.062185,4.049365,0.000024
cosine_itemknn,0.021999,0.127819,0.030119,8.109853,0.000053
pure_svd,0.028351,0.160402,0.061973,5.316310,0.000014


## Metrics std by fold

`If a diff between model metrics less than an std value => there is no significant difference observed`

- For instance, for the serendipity metric there is no such difference between cosine_itemknn and tfidf_itemknn model results

In [23]:
df_metrics.groupby('model').std()[metrics.keys()]

,prec@10,recall@10,map@10,novelty,serendipity
model,,,,,
blend_bm25_pure_svd,0.007259,0.049631,0.023675,0.276969,0.000002
blend_cosine_bm25,0.007176,0.049029,0.015224,0.194977,0.000007
blend_cosine_pure_svd,0.005433,0.038019,0.018824,0.267323,0.000006
blend_cosine_tfidf,0.005165,0.036234,0.011837,0.189257,0.000015
blend_tfidf_bm25,0.007829,0.053007,0.015623,0.177851,0.000010
blend_tfidf_pure_svd,0.005616,0.039193,0.019101,0.265492,0.000009
bm25_itemknn,0.010650,0.069899,0.014546,0.179575,0.000003
cosine_itemknn,0.004735,0.033637,0.007421,0.203701,0.000011
pure_svd,0.008490,0.058740,0.016332,0.307155,0.000002


# Fit best model (bm25_itemknn)

In [24]:
dataset  = Dataset.construct(
    interactions_df=interactions,
    user_features_df=None,
    item_features_df=None
)

In [25]:
model = ImplicitItemKNNWrapperModel(model=BM25Recommender(K=N))
model.fit(dataset)

In [26]:
with open('../data/bm25_itemknn.dill', 'wb') as f:
    dill.dump(model, f)

# Fit model for cold users

In [27]:
popular_model = PopularModel(add_cold=True)
popular_model.fit(dataset)

In [28]:
with open('../data/popular_model.dill', 'wb') as f:
    dill.dump(model, f)

# Offline

In [29]:
with open("../data/bm25_itemknn.dill", "rb") as f:
    model = dill.load(f)

In [30]:
users_ids = interactions['user_id'].unique()

In [31]:
offline_recs = model.recommend(
    users_ids, 
    dataset=dataset, 
    k=20, 
    filter_viewed=False  # False - same items to every user
)

In [32]:
offline_recs = offline_recs.groupby('user_id').agg({'item_id': list})

In [33]:
offline_recs_list = [ None ] * (max(users_ids) + 1)
for i, item in tqdm(offline_recs.iterrows(), total = len(users_ids)):
    offline_recs_list[item.name] = item["item_id"]

  0%|          | 0/962179 [00:00<?, ?it/s]

In [34]:
with open("../data/popular_model.dill", "rb") as f:
    popular_model = dill.load(f)

In [35]:
popular_model_recs = list(popular_model.recommend(
    [0], 
    dataset=dataset, 
    k=20, 
    filter_viewed=False  # False - same items to every user
)['item_id'])

In [36]:
with open('../data/offline_bm25_itemknn.dill', 'wb') as f:
    dill.dump(
        {
            "recs": offline_recs_list,
            "popular_recs": popular_model_recs,
        },
        f,
    )